In [2]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher
import os
from nltk import sent_tokenize
nlp = spacy.load('en_core_web_lg')


In [ ]:
# import sys
# from spacy.symbols import nsubjpass
# 
# # '''
# #  ('((\\bclaim[^\b]+\\b|\\baccept[^\b]+\\b).*(\\bresponsibility\\b))', 'claim'), # perp claimed/accepted responsibility
# #     ('((\\bha[^\b]+\\b)+ *(\\bkilled\\b))', 'kill'), # perp has killed
# #     ('(were +arrested)', 'arrest'), # perp were arrested
# #     ('(blamed) +', 'blame'), # blamed _perp
# #     ("massacred", 'massacre'),
# #     ('kidnapped', 'kidnap'),
# #     ('kidnapped by', 'kidnap'),
# #     ('carried out by', 'carried'),
# #     ('carried out', 'carried'),
# #     ('(killed by)', 'kill'), # killed by. does not perform well
# #     ('(^(we[^\b]+) *destroyed)', 'destroy'),
# #     ('(attacked)', 'attack'),
# #     ("guerrilla['s]? of", '-'),
# #     ("member['s]? of", '-'),
# #     ('threatened', 'threaten'),
# #     ('ambushed', 'ambush')
# # '''
# 
# matching_rules = [
#     ('murder', [{'LEMMA': 'be'}, {'LEMMA': 'murder', 'OP': '*'}, {'POS': 'ADP', 'OP': '*'}]),
#     # ('kidnap', [{'LEMMA': 'be'}, {'LEMMA': 'kidnap'}]),
#     # ('killed', [{'LEMMA': 'be'}, {'LEMMA': 'killed'}]),
#     # ('murdered', [{'ORTH': 'murdered'}]),
#     # ('massacre', [{'ORTH': 'to', 'OP': '!'},{'ORTH': 'massacre'}])
# ]
# 
# output = []
# 
# # Callback for matcher. gets the sentences and start-end position of matches.
# def collect_sents(matcher, doc, i, matches):
#     match_id, start, end = matches[i]
#     span = doc[start : end]
#     sent = span.sent
#     match_ents = {'start': span.start-sent.start, 'end': span.end-sent.start,
#                    'label': doc.vocab.strings[match_id]}
#     output.append({'text': sent.text, 'ents': match_ents})
#     return output
#      
# matcher = Matcher(nlp.vocab)
# 
# [matcher.add(rule[0], collect_sents, rule[1]) for rule in matching_rules] 
# 
# for i,line in enumerate(content[0:5000]):
#     output = []
#     doc = nlp(line)
#     person_list = [ent for ent in doc.ents if ent.label_ == 'PERSON']
#     noun_chunk = []
#     for noun in doc.noun_chunks:
#         noun_chunk.append(noun)
#     matches = matcher(doc)
#     final_output = []
#     if len(output) > 0:    
#         # print(i)
#         rule_label = output[0]['ents']['label']
#         # print(line)
#         # print(len(noun_chunk), noun_chunk)
#         noun_clauses = []
#         for token in doc:
#             if rule_label in token.head.text and token.head.pos_ is not 'NOUN':
#                 # print(token.text, token.pos_)
#                 if 'dobj' in token.dep_:
#                     a = prep_pobj_travel(token)
#                     if a is not None:
#                         # print(a)
#                         # print([x for x in noun_chunk if a in x])
#                         # print()
#                         noun_clauses.extend([x.text for x in noun_chunk if a in x])
#                     else:
#                         noun_clauses.extend([x.text for x in noun_chunk if token in x])
#                 # if 'PROPN' in token.pos_:
#                 #     noun_clauses.extend([x for x in noun_chunk if token in x])
#                 # if token.pos_ == 'NOUN' and 'prep' in [x.dep_ for x in token.children]:
#                 #     for x in token.children:
#                 #         if 'PREP' in token.dep_:
#                 #             noun_clauses.append(corr_np_string(print_noun_recursive(token)))
# 
#             
#         if rule_label in line:
#             for ent in doc.ents:
#                 if ent.label_ == 'PERSON':
#                     noun_clauses.append(ent.text)
#                 # print(ent.text, ent.label_)
#             
#         # rule_pos = line.find(rule_label)
#         # noun_clauses = [(abs(line.find(n) - rule_pos), n) for n in noun_clauses]
#         # print(noun_clauses)
#                     
#         if len(noun_clauses) > 0:
#             print('Matched Rule:', rule_label)
#             print('Matched Line: ', line.upper())
#             print("Victims: ", noun_clauses)
#             print()


In [3]:
input_data = os.path.abspath('merged_project_ips.txt')
with open(input_data, 'r') as f:
    read_content = f.read()
content = [c.replace('\n', ' ').lower() for c in sent_tokenize(read_content)]
read_content = read_content.lower()



In [308]:

output = []

match_pattern_kidnap = [{'LEMMA': 'kidnap', 'POS':'NOUN'}] 
match_pattern_responsibility_claim = [{'LEMMA': 'claim', 'OP': '+'}, {'ORTH': 'responsibility'}]
match_pattern_responsibility_accept = [{'LEMMA': 'accept', 'OP': '+'}, {'ORTH': 'responsibility'}]

def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start : end] # matched span
    sent = span.sent # sentence containing matched span
    # append mock entity for match in displaCy style to matched_sents
    # get the match span by ofsetting the start and end of the span with the
    # start and end of the sentence in the doc
    match_ents = [{'start': span.start-sent.start, 'end': span.end-sent.start,
                   'label': 'MATCH'}]
    output.append({'text': sent.text, 'ents': match_ents })    

from spacy.symbols import nsubjpass
matcher = Matcher(nlp.vocab)

matcher.add('murder_event', collect_sents, match_pattern_responsibility_claim)

matches = matcher(doc)

for o in output:
    # print(o)
    print()
if len(output) > 0:    
    op = output[0]['text']
    
    sent_parse = nlp(op)
    
    for token in sent_parse:
        if token.dep == nsubjpass:
            print(corr_np_string(print_noun_recursive(token)))

In [237]:
import re

regex_rules = [
    ('((\\bclaim[^\b]+\\b|\\baccept[^\b]+\\b).*(\\bresponsibility\\b))', 'claim'), # perp claimed/accepted responsibility
    ('((\\bha[^\b]+\\b)+ *(\\bkilled\\b))', 'kill'), # perp has killed
    ('(were +arrested)', 'arrest'), # perp were arrested
    ('(blamed) +', 'blame'), # blamed _perp
    ("massacred", 'massacre'),
    ('kidnapped', 'kidnap'),
    ('kidnapped by', 'kidnap'),
    ('carried out by', 'carried'),
    ('carried out', 'carried'),
    ('(killed by)', 'kill'), # killed by. does not perform well
    ('(^(we[^\b]+) *destroyed)', 'destroy'),
    ('(attacked)', 'attack'),
    ("guerrilla['s]? of", '-'),
    ("member['s]? of", '-'),
    ('threatened', 'threaten'),
    ('ambushed', 'ambush')
]

In [246]:
def perp_org(text):
    for line in text:
        b = line.lower()
        for i,r in enumerate([x[0] for x in regex_rules]):
            a = re.findall(r, b)
            if len(a) > 0:
                sent = line
                line_parse = nlp(sent)
                orgs = [x for x in line_parse.ents if x.label_ == "ORG"]
                if len(orgs) > 0:
                    orgs = [(x.text, x.start) for x in orgs]
                    reg_pos = line.lower().find(a[0][0])
                    orgs = [(x[0], reg_pos - x[1]) for x in orgs]
                    print(orgs)
                    return min(orgs, key=lambda x: x[1])
               
               
def perp_ind(line):
    b = line.lower()
    for i, r in enumerate([x[0] for x in regex_rules]):
        a = re.findall(r, b)
        if len(a) > 0:
            sent = line
            line_parse = nlp(sent)
            # inds = [x for x in line_parse if 'nsubj' in x.dep_]
            inds = []
            for x in line_parse:
                if x.pos_ == 'VERB' and regex_rules[i][1] in x.text:
                    for c in x.children:
                        if 'obj' in c.dep_:
                            inds.append(corr_np_string(print_noun_recursive(c)))
                    
            if len(inds) > 0:
                print(inds)
                print(r)
                print(line)
                print()
                return inds
            # if len(inds) > 0:
            #     inds = [(x.text, x.start) for x in inds]
            #     reg_pos = line.lower().find(a[0][0])
            #     inds = [(x[0], reg_pos - x[1]) for x in inds]
            #     return min(inds, key=lambda x:x[1])

In [389]:
def print_noun_recursive(token):
    output = [token.text]
    for c in token.children:
        output.extend(print_noun_recursive(c))
    return list(reversed(output))

def corr_np_string(np_chunk):
    return " ".join(np_chunk)

def prep_pobj_travel(token):
    for c in token.children:
        if c.dep_ is 'prep':
            for c_ in c.children:
                if c_.dep_ is 'pobj':
                    return c_

In [68]:
def read_answers():
    answers = 'test_merged_ans.txt'
    with open(answers, 'r') as f:
        answers = f.readlines()
    victim_dict = {}
    curr_id = ""
    is_vic = False
    for ans in answers:
        if 'ID:' in ans:
            is_vic = False
            curr_id = ans.split(':')[1].strip()
        elif 'VICTIM:' in ans or is_vic:
            is_vic = True
            a = ""
            if ':' in ans:
                a = ans.split(':')[1].strip()
            else:
                a = ans.strip()
            
            if curr_id in victim_dict.keys():
                victim_dict[curr_id].append(a)
            else:
                victim_dict[curr_id] = [a]
        elif ':' in ans:
            is_vic = False
    victim_dict = {key: [x for x in value if len(x.strip()) > 1 or x == '-'] for (key, value) in victim_dict.items()}
    return victim_dict

victim_dictionary = read_answers()

In [69]:
import re
def read_files():
    text_files = os.path.abspath('test_merged_art.txt')
    id_match = re.compile('[A-Z]{3}[1-9]*-[A-Z]{3}[1-9]*-[0-9]{4}')
    id_stuff = re.compile('\(.*?\)')
    start_regex = re.compile('.*\[.*?\]')
    with open(text_files, 'r') as f:
        content = f.readlines()
        
    content = [c.replace('\n', ' ').strip() for c in content]
    
    file_dict = {}    
    
    curr_file_id = ""
    file = []
    for line in content:
        if id_match.match(line):
            if len(file) > 0:
                file = " ".join(file)
                file_dict[curr_file_id] = file.strip()
                file = []
            stuff = id_stuff.findall(line)
            for s in stuff:
                line = line.replace(s, '')
            curr_file_id = line.strip()
        else:
            if start_regex.findall(line):
                line = line.replace(start_regex.findall(line)[0], '')
            file.append(line.strip())
    return file_dict

file_dict = read_files()

In [90]:
output = []
def collect_sents(matcher, doc, i, matches):
        match_id, start, end = matches[i]
        span = doc[start : end]
        sent = span.sent
        match_ents = {'start': span.start-sent.start, 'end': span.end-sent.start,
                       'label': doc.vocab.strings[match_id]}
        output.append({'text': sent.text, 'ents': match_ents})
        return output


def get_victim(line_l, nlp, Matcher):
    global output
    output = []
    line = line_l.lower().replace('\n', ' ')
    
    keep_char_regex = re.compile('^[\W\s]')
    line = keep_char_regex.sub('', line)
    matching_rules = [
        ('murder_passive', [{'LEMMA': 'be'}, {'LEMMA': 'murder', 'OP': '*'}, {'POS': 'ADP', 'OP': '*'}]),
        ('kidnap', [{'LEMMA': 'be'}, {'LEMMA': 'kidnap'}]),
        ('killed', [{'LEMMA': 'be'}, {'LEMMA': 'killed'}]),
        ('murder_active', [{'LEMMA': 'be', 'OP': '!'}, {'ORTH': 'murdered'}]),
        ('massacre', [{'ORTH': 'to', 'OP': '!'}, {'ORTH': 'massacre'}]),
        ('wounded_passive', [{'LEMMA': 'be'}, {'LEMMA': 'wound'}])
    ]

    matcher = Matcher(nlp.vocab)
    [matcher.add(rule[0], collect_sents, rule[1]) for rule in matching_rules]

    doc = nlp(line)
    noun_chunk = []
    for noun in doc.noun_chunks:
        noun_chunk.append(noun.text)
    matcher(doc)
    final_nouns = []
    if len(output) > 0:
        rule_label = output[0]['ents']['label']
        noun_clauses = []

        for i,token in enumerate(doc):
            if 'jesuit' in token.text and 'priests' in doc[i+1].text:
                final_nouns.append('JESUIT PRIESTS')
            elif 'priest' in token.text:
                final_nouns.append('PRIESTS')
            elif 'jesuit' in token.text:
                final_nouns.append('JESUIT')
            else:
                if 'murder' in token.text:
                    if rule_label == 'murder_active':
                        noun_clauses.extend([x.text for x in token.rights])
                if 'wounded' in token.text:
                    noun_clauses.extend([x.text for x in token.rights])
                
        if 'murder' in line:
            for ent in doc.ents:
                if ent.label_ == 'PERSON':
                    for x in noun_chunk:
                        if ent.text in x or x in ent.text:
                            if len(x) > len(ent.text):
                                final_nouns.append(x)
                            else:
                                final_nouns.append(ent.text)

        if 'kidnap' in line:
            for ent in doc.ents:
                if ent.label_ == 'PERSON':
                    for x in noun_chunk:
                        if ent.text in x or x in ent.text:
                            if len(x) > len(ent.text):
                                final_nouns.append(x)
                            else:
                                final_nouns.append(ent.text)

        if 'kill' in line:
            for ent in doc.ents:
                if ent.label_ == 'PERSON':
                    for x in noun_chunk:
                        if ent.text in x or x in ent.text:
                            if len(x) > len(ent.text):
                                final_nouns.append(x)
                            else:
                                final_nouns.append(ent.text)
                                
        if 'massacre' in line:
            for ent in doc.ents:
                if ent.label_ == 'PERSON':
                    for x in noun_chunk:
                        if ent.text in x or x in ent.text:
                            if len(x) > len(ent.text):
                                final_nouns.append(x)
                            else:
                                final_nouns.append(ent.text)
        
        for n in noun_clauses:
            for x in noun_chunk:
                if n in x:
                    final_nouns.append(x)

        final_nouns = [x.upper().strip() for x in final_nouns]

        if len(final_nouns) > 0:
            return final_nouns
    return ['-']


In [ ]:
victim_ouputs = {}
for file in file_dict:
    vics = []
    for line in sent_tokenize(file_dict[file]):
        v = get_victim(line, nlp, Matcher)
        vics.extend(v)
    victim_ouputs[file] = list(set(vics))

for key, value in victim_dictionary.items():
    if key in victim_ouputs.keys():
        print(value)
        print(victim_ouputs[key])
        print()

In [105]:
for file in file_dict:
    vics = []
    for line in sent_tokenize(file_dict[file]):
		annotations = annotator.getAnnotations(line)
		print(annotations)
	

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-105-0911a33560f1>, line 4)